<a href="https://colab.research.google.com/github/zhangxs131/NMT-example/blob/main/hf_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##translation

使用huggingface transformer完成的translation

*    数据集 OPUS books dataset en-fr
*    模型  t5-small

In [1]:
!pip install transformers datasets

     |████████████████████████████████| 4.0 MB 10.4 MB/s 
     |████████████████████████████████| 325 kB 63.9 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 6.6 MB 36.3 MB/s 
     |████████████████████████████████| 596 kB 43.3 MB/s 
     |████████████████████████████████| 895 kB 51.8 MB/s 
     |████████████████████████████████| 136 kB 52.6 MB/s 
     |████████████████████████████████| 212 kB 56.4 MB/s 
     |████████████████████████████████| 1.1 MB 47.8 MB/s 
     |████████████████████████████████| 127 kB 57.5 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 45.7 MB/s 
     |████████████████████████████████| 144 kB 46.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstallin

In [3]:
from datasets import load_dataset

books=load_dataset('opus_books','en-fr')

books=books['train'].train_test_split(test_size=0.2)
books['train'][0]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to /root/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

{'id': '65845',
 'translation': {'en': '"Oh, don\'t let\'s go in there!',
  'fr': '« Oh ! n’entrons pas la ! dis-je.'}}

In [ ]:
#进行数据处理

from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq

tokenizer=AutoTokenizer.from_pretrained('t5-small')

source_lang='en'
target_lang='fr'
prefix='translation English to French: '

def preprocess_function(examples):
  inputs=[prefix+example[source_lang] for example in examples['translation']]
  targets=[example[target_lang] for example in examples['translation']]
  model_inputs=tokenizer(inputs,max_length=128,truncation=True)

  with tokenizer.as_target_tokenizer():
    label=tokenizer(targets,max_length=128,truncation=True)
  model_inputs['labels']=label['input_ids']
  return model_inputs

tokenized_books=books.map(preprocess_function,batched=True)

In [ ]:
#train
from transformers import AutoModelForSeq2SeqLM,Seq2SeqTrainingArguments, Seq2SeqTrainer

model=AutoModelForSeq2SeqLM.from_pretrained('t5-small')

data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()